# Question 1: datetime Fundamentals and Time Series Indexing

This question focuses on datetime handling and time series indexing using patient vital signs data.

## Setup

In [224]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os

# Set random seed for reproducibility
np.random.seed(42)

# Set plotting style
plt.style.use('default')
sns.set_style('whitegrid')

# Create output directory
os.makedirs('output', exist_ok=True)

## Part 1.1: Load and Explore Data

**Note:** This dataset contains realistic healthcare data characteristics:
- **200 patients** with daily vital signs over 1 year
- **Missing visits**: Patients miss approximately 5% of scheduled visits (realistic!)
- **Different start dates**: Not all patients start monitoring on January 1st (some join later)
- When selecting data by date ranges, you may find that some patients don't have data for certain periods - this is expected and realistic

In [225]:
# Load patient vital signs data
patient_vitals = pd.read_csv('data/patient_vitals.csv')

print("Patient vitals shape:", patient_vitals.shape)
print("Patient vitals columns:", patient_vitals.columns.tolist())

# Display sample data
print("\nPatient vitals sample:")
print(patient_vitals.head())
print("\nData summary:")
print(patient_vitals.describe())

# Check date range and missing data patterns
print(f"\nDate range: {patient_vitals['date'].min()} to {patient_vitals['date'].max()}")
print(f"Unique patients: {patient_vitals['patient_id'].nunique()}")
print(f"Total records: {len(patient_vitals)}")
print(f"Expected records (200 patients × 365 days): {200 * 365:,}")
print(f"Missing visits: ~{200 * 365 - len(patient_vitals):,} records")

Patient vitals shape: (18250, 7)
Patient vitals columns: ['date', 'patient_id', 'temperature', 'heart_rate', 'blood_pressure_systolic', 'blood_pressure_diastolic', 'weight']

Patient vitals sample:
         date patient_id  temperature  heart_rate  blood_pressure_systolic  \
0  2023-01-01      P0001    98.389672          71                      119   
1  2023-01-02      P0001    98.492046          67                      117   
2  2023-01-03      P0001    98.790163          70                      113   
3  2023-01-04      P0001    98.635781          74                      117   
4  2023-01-05      P0001    98.051660          67                      118   

   blood_pressure_diastolic     weight  
0                        84  68.996865  
1                        82  67.720215  
2                        78  67.846825  
3                        82  67.693993  
4                        83  68.228852  

Data summary:
        temperature    heart_rate  blood_pressure_systolic  \
count  182

## Part 1.2: datetime Operations

**TODO: Perform datetime operations**

In [226]:
# TODO: Convert date column to datetime

print(patient_vitals.shape)
patient_vitals['date'] = pd.to_datetime(patient_vitals['date'])

# print(patient_vitals['date'])

# TODO: Set datetime column as index
patient_vitals = patient_vitals.set_index('date')

# print(patient_vitals)

# TODO: Extract year, month, day components from datetime index
patient_vitals['year'] = patient_vitals.index.year  # Extract from index

# print(patient_vitals['year'])
patient_vitals['month'] = patient_vitals.index.month  # Extract from index

# print(patient_vitals['month'])
patient_vitals['day'] = patient_vitals.index.day   # Extract from index

# print(patient_vitals['day'])

# TODO: Calculate time differences (e.g., days since first measurement)
# Note: Since patients start at different times, calculate days_since_start per patient
# Hint: To use groupby on the 'date' column, temporarily reset the index, then set it back
# Example: patient_vitals_reset = patient_vitals.reset_index()
#          Use groupby('patient_id')['date'].transform(lambda x: (x - x.min()).dt.days)
#          Or use groupby('patient_id').apply() to calculate days from each patient's first date
#          Then: patient_vitals = patient_vitals_reset.set_index('date')
patient_vitals_reset = patient_vitals.reset_index()

patient_vitals_reset['days_since_start'] = patient_vitals_reset.groupby("patient_id")["date"].transform(lambda x: (x - x.min()).dt.days)  # Calculate from each patient's start date

patient_vitals = patient_vitals_reset.set_index('date')

print(patient_vitals['days_since_start'])

print(patient_vitals.columns)

# TODO: Create business day ranges for clinic visit schedules
# patient_vitals_min_max = patient_vitals.reset_index().groupby('patient_id')['date'].agg(['min', 'max'])

patient_vitals_reset_2 = patient_vitals.reset_index()
clinic_dates = pd.bdate_range(start = min(patient_vitals_reset_2["date"]), end = max(patient_vitals_reset_2["date"]))
print(f'clinic dates: {clinic_dates}')

# patient_vitals_min_max.columns = ['start_date', 'end_date']
# print(patient_vitals_min_max)

#def clinic_schedule(row):
  #  return pd.bdate_range(start=row['start_date'], end=row['end_date'])
#clinic_dates = patient_vitals_min_max.apply(clinic_schedule, axis = 1)
# print(clinic_dates)



# patient_vitals_reset.groupby("patient_id")["date"].agg(start_date = "min", end_date = "max").reset_index()

# print(patient_vitals_reset)

# TODO: Create date ranges with different frequencies
daily_range = pd.date_range(start = min(patient_vitals_reset_2["date"]), end = max(patient_vitals_reset_2["date"]), freq='D')  # Daily monitoring schedule
print(daily_range)
weekly_range = pd.date_range(start = min(patient_vitals_reset_2["date"]), end = max(patient_vitals_reset_2["date"]), freq='W-MON')  # Weekly lab test schedule (Mondays)
print(weekly_range)
monthly_range = pd.date_range(start = min(patient_vitals_reset_2["date"]), end = max(patient_vitals_reset_2["date"]), freq='MS')  # Monthly checkup schedule
print(monthly_range)
# TODO: Use date ranges to analyze visit patterns
# Check how many patient visits occurred on clinic business days vs weekends
patient_dates_set = set(patient_vitals.index.date)
# print(patient_dates_set)
clinic_dates_set = set(clinic_dates.date)
# print(clinic_dates_set)
visits_on_clinic_days = len(patient_dates_set & clinic_dates_set)
visits_on_weekends = len(patient_dates_set) - visits_on_clinic_days
print(f"Visits on clinic business days: {visits_on_clinic_days}")
print(f"Visits on weekends: {visits_on_weekends}")
print(f"Total unique visit dates: {len(patient_dates_set)}")

# TODO: Save results as 'output/q1_datetime_analysis.csv'
# Create a DataFrame with datetime analysis results including:
# - date (datetime index or column)
# - year, month, day (extracted from datetime)
# - days_since_start (calculated time differences)
# - patient_id
# - At least one original column (e.g., temperature, heart_rate)
# Note: When saving to CSV with index=False, you'll need to convert the index to a column first
# Example structure:
datetime_analysis = patient_vitals[['patient_id', 'year', 'month', 'day', 'days_since_start', 'temperature']].copy().reset_index()
datetime_analysis.to_csv('output/q1_datetime_analysis.csv', index=False)

(18250, 7)
date
2023-01-01      0
2023-01-02      1
2023-01-03      2
2023-01-04      3
2023-01-05      4
             ... 
2023-12-27    360
2023-12-28    361
2023-12-29    362
2023-12-30    363
2023-12-31    364
Name: days_since_start, Length: 18250, dtype: int64
Index(['patient_id', 'temperature', 'heart_rate', 'blood_pressure_systolic',
       'blood_pressure_diastolic', 'weight', 'year', 'month', 'day',
       'days_since_start'],
      dtype='object')
clinic dates: DatetimeIndex(['2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05',
               '2023-01-06', '2023-01-09', '2023-01-10', '2023-01-11',
               '2023-01-12', '2023-01-13',
               ...
               '2023-12-18', '2023-12-19', '2023-12-20', '2023-12-21',
               '2023-12-22', '2023-12-25', '2023-12-26', '2023-12-27',
               '2023-12-28', '2023-12-29'],
              dtype='datetime64[ns]', length=260, freq='B')
DatetimeIndex(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04',
   

## Part 1.3: Time Zone Handling

**TODO: Handle time zones**

In [227]:
# TODO: Create timezone-aware datetime (for multi-site clinical trials)
utc_time = pd.Timestamp.now(tz='UTC')  # Current time in UTC
print(utc_time)
eastern_time = utc_time.tz_convert('US/Eastern') # Convert to US Eastern
print(eastern_time)

# TODO: Convert between different timezones
# Create timezone-aware DataFrame from patient_vitals
patient_vitals_tz = patient_vitals.index.tz_localize('UTC')  # Localize to UTC
print(patient_vitals_tz)
patient_vitals_tz_eastern = patient_vitals_tz.tz_convert('US/Eastern')  # Convert to Eastern time
print(patient_vitals_tz_eastern)

# TODO: Handle daylight saving time transitions
# Create datetime that spans DST transition
# Note: Using UTC avoids DST ambiguity issues - UTC has no daylight saving time
# Best practice: Store data in UTC, convert to local timezones only when needed
dst_date_utc = pd.Timestamp('2023-03-12 10:00:00', tz='UTC')  # UTC time avoids DST issues
print(dst_date_utc)
dst_time_eastern = dst_date_utc.tz_convert('US/Eastern')  # Convert UTC to Eastern
print(dst_time_eastern)

# TODO: Document timezone operations
# Create a report string with the following sections:
# 1. Original timezone: Describe what timezone your original data was in (or if it was naive)
# 2. Localization method: Explain how you localized the data (e.g., tz_localize('UTC'))
# 3. Conversion: Describe what timezone you converted to (e.g., 'US/Eastern')
# 4. DST handling: Document any issues or observations about daylight saving time transitions
#    Note: Explain why using UTC as the base timezone avoids DST ambiguity issues
# 5. Example: Show at least one example of a datetime before and after conversion
# Minimum length: 200 words
timezone_report = """
My original data from the dataframe of patient_vitals was in naive datetime. 
I localized the data using tz_localize('UTC') in order to add the timezone of UTC to the naive datetime in patient_vitals. 
I then converted the timezone in patient_vitals from 'UTC' to 'US/Eastern'. 
An issue that I encountered with DST is that when we try to convert to the timezone of US/Eastern, issues can arise due to daylight savings 
that happens with US/Eastern time. On the second Sunday of March every year, clocks skip one hour and go directly from 2:00 a.m. to
3 a.m. This means that 2:30 a.m. on this day is an invalid time, and this can cause ambiguity if we're trying to convert to US/Eastern time, but
the given time on the second Sunday of March we're trying to convert does not exist in US/Eastern time due to DST. Similarly, on the first Sunday every November, at 2 a.m. 
clocks move back one hour to 1 a.m. If we're tying to convert a time on the first Sunday of November to US/Eastern time, ambiguity can arise due to the 
repeated hour. Using UTC as the base timezone avoids these issues with DST because UTC doesn't observe DST, which means we have an unambiguous reference point. 
This means that pandas will be able to correctly calculate the local time for the new timezone and properly adjust for DST changes without ambiguity. 

Example: 
We have 
dst_date_utc = pd.Timestamp('2023-03-12 10:00:00', tz='UTC')  # UTC time avoids DST issues
print(dst_date_utc)
dst_time_eastern = dst_date_utc.tz_convert('US/Eastern')  # Convert UTC to Eastern
print(dst_time_eastern)

When we convert '2023-03-12 10:00:00' in UTC to US/Eastern, we get '2023-03-12 06:00:00-04:00'. We see that pandas takes into account that an hour is skipped 
on the Sunday of March for US/Eastern, causing US/Eastern to to trail UTC by 4 hours as part of DST.
"""

# TODO: Save results as 'output/q1_timezone_report.txt'
with open('output/q1_timezone_report.txt', 'w') as f:
    f.write(timezone_report)



2025-11-13 00:27:50.506916+00:00
2025-11-12 19:27:50.506916-05:00
DatetimeIndex(['2023-01-01 00:00:00+00:00', '2023-01-02 00:00:00+00:00',
               '2023-01-03 00:00:00+00:00', '2023-01-04 00:00:00+00:00',
               '2023-01-05 00:00:00+00:00', '2023-01-06 00:00:00+00:00',
               '2023-01-07 00:00:00+00:00', '2023-01-08 00:00:00+00:00',
               '2023-01-09 00:00:00+00:00', '2023-01-10 00:00:00+00:00',
               ...
               '2023-12-22 00:00:00+00:00', '2023-12-23 00:00:00+00:00',
               '2023-12-24 00:00:00+00:00', '2023-12-25 00:00:00+00:00',
               '2023-12-26 00:00:00+00:00', '2023-12-27 00:00:00+00:00',
               '2023-12-28 00:00:00+00:00', '2023-12-29 00:00:00+00:00',
               '2023-12-30 00:00:00+00:00', '2023-12-31 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='date', length=18250, freq=None)
DatetimeIndex(['2022-12-31 19:00:00-05:00', '2023-01-01 19:00:00-05:00',
               '2023-01-02 19:

## Submission Checklist

Before moving to Question 2, verify you've created:

- [ ] `output/q1_datetime_analysis.csv` - datetime analysis results
- [ ] `output/q1_timezone_report.txt` - timezone handling report
